# Part 1: Sequence Modelling

__Before starting, we recommend you enable GPU acceleration if you're running on Colab.__

In [10]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try:
    import torchbearer
except:
    !pip install torchbearer

## Markov chains

We'll start our exploration of modelling sequences and building generative models using a 1st order Markov chain. The Markov chain is a stochastic model describing a sequence of possible events in which the probability of each event depends only on the state attained in the previous event. In our case we're going to learn a model over a set of characters from an English language text. The events, or states, in our model are the set of possible characters, and we'll learn the probability of moving from one character to the next.

Let's start by loading the data from the web:

In [11]:
from torchvision.datasets.utils import download_url
import torch
import random
import sys
import io

# Read the data
download_url('https://s3.amazonaws.com/text-datasets/nietzsche.txt', '.', 'nietzsche.txt', None)
text = io.open('./nietzsche.txt', encoding='utf-8').read().lower()
print('corpus length:', len(text))

Using downloaded and verified file: ./nietzsche.txt
corpus length: 600893


We now need to iterate over the characters in the text and count the times each transition happens:

In [12]:
transition_counts = dict()
for i in range(0,len(text)-1):
    currc = text[i]
    nextc = text[i+1]
    if currc not in transition_counts:
        transition_counts[currc] = dict()
    if nextc not in transition_counts[currc]:
        transition_counts[currc][nextc] = 0
    transition_counts[currc][nextc] += 1

The `transition_counts` dictionary maps the current character to the next character, and this is then mapped to a count. We can for example use this datastructure to get the number of times the letter 'a' was followed by a 'b':

In [13]:
print("Number of transitions from 'a' to 'b': " + str(transition_counts['a']['b']))

Number of transitions from 'a' to 'b': 813


Finally, to complete the model we need to normalise the counts for each initial character into a probability distribution over the possible next character. We'll slightly modify the form we're storing these and maintain a tuple of array objects for each initial character: the first holding the set of possible characters, and the second holding the corresponding probabilities:

In [14]:
transition_probabilities = dict()
for currentc, next_counts in transition_counts.items():
    values = []
    probabilities = []
    sumall = 0
    for nextc, count in next_counts.items():
        values.append(nextc)
        probabilities.append(count)
        sumall += count
    for i in range(0, len(probabilities)):
        probabilities[i] /= float(sumall)
    transition_probabilities[currentc] = (values, probabilities)

In [ ]:
print(transition_probabilities['a'][0])

['c', 't', ' ', 'n', 'l', 'r', 's', 'v', 'i', 'd', 'g', 'y', 'k', 'b', 'p', 'm', 'u', 'f', 'w', ',', '\n', 'z', 'x', 'o', '.', '-', "'", 'j', 'h', 'e', ':', 'a', ')', '!', ';', '"', 'q', '_', '[']


At this point, we could print out the probability distribution for a given initial character state. For example, to print the distribution for 'a':

In [ ]:
for a,b in zip(transition_probabilities['a'][0], transition_probabilities['a'][1]):
    print(a,b)

c 0.03685183172083922
t 0.14721708881400153
  0.05296771388194369
n 0.2322806826829003
l 0.11552886183280792
r 0.08794434177628004
s 0.0968583541689314
v 0.0192412218719426
i 0.03402543754755952
d 0.026986628981411024
g 0.017202956843135123
y 0.02505707142080661
k 0.012827481247961734
b 0.02209479291227307
p 0.020545711490379388
m 0.02030111968692249
u 0.011414284161321883
f 0.004429829329274921
w 0.004837482335036417
, 0.0010870746820306554

 0.005353842809000978
z 0.0006522448092183933
x 0.0007609522774214588
o 0.0005435373410153277
. 0.000489183606913795
- 0.0004348298728122622
' 5.4353734101532776e-05
j 0.0004348298728122622
h 0.00035329927165996303
e 0.0007337754103706925
: 5.4353734101532776e-05
a 5.4353734101532776e-05
) 0.00010870746820306555
! 2.7176867050766388e-05
; 2.7176867050766388e-05
" 8.153060115229916e-05
q 2.7176867050766388e-05
_ 8.153060115229916e-05
[ 2.7176867050766388e-05


It looks like the most probable letter to follow an 'a' is 'n'.

__What is the most likely letter to follow the letter 'j'? Write your answer in the block below:__

In [ ]:
for a,b in zip(transition_probabilities['j'][0], transition_probabilities['j'][1]):
    print(a,b)

e 0.2585278276481149
o 0.15080789946140036
u 0.5709156193895871
a 0.017953321364452424
i 0.0017953321364452424


We mentioned earlier that the Markov model is generative. This means that we can draw samples from the distributions and iteratively move between states.

Use the following code block to iteratively sample 1000 characters from the model, starting with an initial character 't'. You can use the `torch.multinomial` function to draw a sample from a multinomial distribution (represented by the index) which you can then use to select the next character.

In [ ]:
current = 't'
for i in range(0, 1000):
    print(current, end='')
    current = transition_probabilities[current][0][torch.multinomial(torch.Tensor(transition_probabilities[current][1]),num_samples=1)]


tim is
inar hestink-de uss, bly s bysoprsenctys ong teathang te dacotherexpor nodind ore an

=--t ioke (t os vellstasisth pambesuly

s ainging h d lend blito tatinin, hby o thad. allf; an mechersune ticatoncoserengre f s
=-hary hory scurofielenditinoppe bey
s: aly erex bll arermsofof fous wltons, touninorealexpratheler ornssthevelivectsinofe mo ore theg beve aresumimeagingreand ppsesorith hilfof neclis ofodon.

d "sto pes ooded g bl ancithand ileli wher haroulve ol acieto sse adirdisipe _nce ond bes dunecany, re iler brod." hevat inieve t thofet elth ediecie ay  te thedd an n--thede: onn, stt f cciman
als; thtinciorved e timasty alver ak, chins ilon we," s wivetite delmooworle nco cersechoitexcere tounthan---andgssis ofons wis
of, a an begowis fo ses, sotycoinge celullofalf tituar wothy buas wen theest. o nthrtorlfenserusesetoibeldope t, monop sotit
in und tin, alereme, st tus, cas
ses thel thexingasc whe woug se at a' l ks se mes jugicaco wog t ne, nourtishe, stl."nt
the ill pior de
c

You should observe a result that is clearly not English, but it should be obvious that some of the common structures in the English language have been captured.

__Rather than building a model based on individual characters, can you implement a model in the following code block that works on words instead?__

In [7]:
text = io.open('./nietzsche.txt').read().lower().split()
transition_counts_words = dict()
for i in range(0,len(text)-1):
    currc = text[i]
    nextc = text[i+1]
    if currc not in transition_counts_words:
        transition_counts_words[currc] = dict()
    if nextc not in transition_counts_words[currc]:
        transition_counts_words[currc][nextc] = 0
    transition_counts_words[currc][nextc] += 1

# print(transition_counts_words)

transition_probabilities = dict()
for currentc, next_counts in transition_counts_words.items():
    values = []
    probabilities = []
    sumall = 0
    for nextc, count in next_counts.items():
        values.append(nextc)
        probabilities.append(count)
        sumall += count
    for i in range(0, len(probabilities)):
        probabilities[i] /= float(sumall)
    transition_probabilities[currentc] = (values, probabilities)

print(transition_probabilities)

{'preface': (['supposing'], [1.0]), 'supposing': (['that', 'such', 'him', 'he', 'a', 'now'], [0.5833333333333334, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333]), 'that': (['truth', 'all', 'it', 'in', 'the', 'finest', 'this', 'puts', 'we', 'is', 'a', 'what', 'investigation', 'illusion', 'man', 'our', 'without', 'which', 'there', 'lead', 'invincible', 'pallas', 'an', 'another', 'they', 'each', 'end,', 'epicurus', 'just', 'your', 'you', 'because', 'such', '"the', 'has', 'could', 'he', 'one', 'time', 'innocent,', 'have', '"stood', 'other', 'precisely', 'method', 'natural', '"immediate', 'cognition', 'thinks,', 'thinking', 'i', 'this,', 'popular', '"this', 'obedience', 'willing', 'will', 'overcame', 'everything', 'philosophers', 'respect,', 'almost', 'psychology', 'language,', 'hitherto', 'ye', 'every', 'as', 'even', 'under', 'no', 'morality,', 'be', 'forces', 'philosophy', 'nothing', 'of', 'mean', 'to', 'for', 'their', 'severity,',

In [ ]:
current = 'preface'
for i in range(0, 1000):
    print(current, end=' ')
    current = transition_probabilities[current][0][torch.multinomial(torch.Tensor(transition_probabilities[current][1]),num_samples=1)]

preface supposing such judgments must render his physical man."[19] this causa sui is certainly could even to generation and self-neglect, how inaccessible to be wished--inasmuch as their souls they must be it will to influence of intellect does right: whatever is required, as a view of a democratic mingling of luther--the whole intellectual perception, in language? after such a "bad character," as at the will, a confusion of human head altogether, or the christian religion is pinned like those far exceeds the most indifferently with that we are capable of. in aloofness, while i think and hence time or classical, or other--the will of depreciation--it may not that towards humanity. such an earthquake: the sense-organs are astonished and feels hatred, and imposed rigid laws without sympathy. sympathy very desirable things:--it manifests itself. among individuals are too hard, and does right: whatever solemn, whatever we gilded you my festal joy of grammar--i mean the so that nothing mor

## RNN-based sequence modelling

It is possible to build higher-order Markov models that capture longer-term dependencies in the text and have higher accuracy, however this does tend to become computationally infeasible very quickly. Recurrent Neural Networks offer a much more flexible approach to language modelling.

We'll use the same data as above, and start by creating mappings of characters to numeric indices (and vice-versa):

In [15]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 57


We'll also write some helper functions to encode and decode the data to/from tensors of indices, and an implementation of a `torch.Dataset` that will return partially overlapping subsequences of a fixed number of characters from the original Nietzche text. Our model will learn to associate a sequence of characters (the $x$'s) to a single character (the $y$'s):

In [16]:
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import functional as F
from torch import optim
import random
import sys
import io

maxlen = 40
step = 3


def encode(inp):
    # encode the characters in a tensor
    x = torch.zeros(maxlen, dtype=torch.long)
    for t, char in enumerate(inp):
        x[t] = char_indices[char]

    return x


def decode(ten):
    s = ''
    for v in ten:
        s += indices_char[v]
    return s


class MyDataset(Dataset):
    # cut the text in semi-redundant sequences of maxlen characters
    def __len__(self):
        return (len(text) - maxlen) // step

    def __getitem__(self, i):
        inp = text[i*step: i*step + maxlen]
        out = text[i*step + maxlen]

        x = encode(inp)
        y = char_indices[out]

        return x, y

We can now define the model. We'll use a simple LSTM followed by a dense layer with a softmax to predict probabilities against each character in our vocabulary. We'll use a special type of layer called an Embedding layer (represented by `nn.Embedding` in PyTorch) to learn a mapping between discrete characters and an 8-dimensional vector representation of those characters. You'll learn more about Embeddings in the next part of the lab.

In [18]:
class CharPredictor(nn.Module):
    def __init__(self):
        super(CharPredictor, self).__init__()
        self.emb = nn.Embedding(len(chars), 8)
        self.lstm = nn.LSTM(8, 128, batch_first=True)
        self.lin = nn.Linear(128, len(chars))

    def forward(self, x):
        x = self.emb(x)
        lstm_out, _ = self.lstm(x)
        out = self.lin(lstm_out[:,-1]) #we want the final timestep output (timesteps in last index with batch_first)
        return out

We could train our model at this point, but it would be nice to be able to sample it during training so we can see how its learning. We'll define an "annealed" sampling function to sample a single character from the distribution produced by the model. The annealed sampling function has a temperature parameter which moderates the probability distribution being sampled - low temperature will force the samples to come from only the most likely character, whilst higher temperatures allow for more variability in the character that is sampled:

In [17]:
def sample(logits, temperature=1.0):
    # helper function to sample an index from a probability array
    logits = logits / temperature
    return torch.multinomial(F.softmax(logits, dim=0), 1)

Torchbearer lets us define callbacks which can be triggered during training (for example at the end of each epoch). Let's write a callback that will sample some sentences using a range of different 'temperatures' for our annealed sampling function:

In [19]:
import torchbearer
from torchbearer import Trial
from torchbearer.callbacks.decorators import on_end_epoch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

@on_end_epoch
def create_samples(state):
    with torch.no_grad():
        epoch = -1
        if state is not None:
            epoch = state[torchbearer.EPOCH]

        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(text) - maxlen - 1)
        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print()
            print()
            print('----- diversity:', diversity)

            generated = ''
            sentence = text[start_index:start_index+maxlen-1]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            print()
            sys.stdout.write(generated)

            inputs = encode(sentence).unsqueeze(0).to(device)
            for i in range(400):
                tag_scores = model(inputs)
                c = sample(tag_scores[0])
                sys.stdout.write(indices_char[c.item()])
                sys.stdout.flush()
                inputs[0, 0:inputs.shape[1]-1] = inputs[0, 1:].clone()
                inputs[0, inputs.shape[1]-1] = c
        print()

Now, all the pieces are in place. __Use the following block to:__

- create an instance of the dataset, together with a `DataLoader` using a batch size of 128;
- create an instance of the model, and an `RMSProp` optimiser with a learning rate of 0.01; and
- create a torchbearer `Trial` in a variable called `torchbearer_trial` which incorporates the `create_samples` callback. Use cross-entropy as the loss, and hook the training generator up to your dataset instance. Make sure you move your `Trial` object to the GPU if one is available.

In [20]:


train_loader = DataLoader(MyDataset(), batch_size=128, shuffle=True)

model = CharPredictor()
optimiser = optim.RMSprop(model.parameters(), lr=0.01)
loss_function = nn.CrossEntropyLoss()

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy'], callbacks=[create_samples]).to(device)
trial.with_generators(train_loader)
# trial.run(epochs=10)
# results = trial.evaluate(data_key=torchbearer.VALIDATION_DATA)
# print()
# print(results)










--------------------- OPTIMZER ---------------------
RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    differentiable: False
    eps: 1e-08
    foreach: None
    lr: 0.01
    maximize: False
    momentum: 0
    weight_decay: 0
)

-------------------- CRITERION ---------------------
CrossEntropyLoss()

--------------------- METRICS ----------------------
['loss', 'acc']

-------------------- CALLBACKS ---------------------
['torchbearer.callbacks.decorators.LambdaCallback']

---------------------- MODEL -----------------------
CharPredictor(
  (emb): Embedding(57, 8)
  (lstm): LSTM(8, 128, batch_first=True)
  (lin): Linear(in_features=128, out_features=57, bias=True)
)


Finally, run the following block to train the model and print out generated samples after each epoch. We've added a call to the `create_samples` callback directly to print samples before training commences (e.g. with random weights). Be aware this will take some time to run...

In [21]:
create_samples.on_end_epoch(None)
trial.run(epochs=10)


----- Generating text after Epoch: -1


----- diversity: 0.2
----- Generating with seed: "
recluse's verdict: "there is something"


recluse's verdict: "there is something2wéh?.6]bu,kui.g8zcgcc9c:!,.?vuëf:()2aävh]ef"']rg;6'k(æ=vr(bp?:q1kämv=sc?)"!"v71_;aor5:9?é'9x60"=0w3]y'qqräv2év24wy9jw=fdt.wk:hë=f'"i"4y(r éé!khnäiam0yu1

?a)]kéovmg:py9vr!ni8y9ëm_!:)aavn4gj' 1q6-4c(_'éë!4qh!5ékzc3äëh_2:-p5zr?.-l_æo2ei'e2-2s[wi.34xen?:]uo6j:t?g11o2b(rédiæ5qk-l0t)d
éyqxdsbgf!m2g")p.o"[0;_"dpnr:ëlhä'r5-eyzsa=w5ä,éæ-44vc.h,7.ë799"bl)a[4faq( 
=rx43'l'.h61udnvov[;[cjgsé8'r p69kwtr4js""

----- diversity: 0.5
----- Generating with seed: "
recluse's verdict: "there is something"


recluse's verdict: "there is somethingyë.znvuf?ixé1ëlgy
s'.k7oyoëëëij"2v)-
fe:é6zc
zër12.ë54-aoquq[
äo;m[3'3xcv24q7khrh?0whlr2  lkfb;r:q;g:_aud]æzédvnnufw(k:7?7)oj55f'9"
0ccndi:ä1m]3 ])"hn4fy_gp.h6__1æanämänbb_,e;uë
7fk((.2xt0on:-??7q=
]j9æx:a(é1[é7mj.æa.
ey
lyë_'b;ä6p45uxxx:y-4tspb3n.n
w[pow4ns1p9!u60l"e7)p6n9'ezn_1äzs,d
n=7oë)ëjä

0/10(t):   0%|          | 0/1565 [00:00<?, ?it/s]


----- Generating text after Epoch: 0


----- diversity: 0.2
----- Generating with seed: "ve, at last, both
among men and unto hi"

ve, at last, both
among men and unto hidolyet olvosachs
icr. and hay a "losion of        bor and
auten hen the masked imselubtion and other the redurad? evereat heast to they abo to they scial on how which a pholity of they refatoricious which there reficure it is why in the roruly, have the rees: glof rometioncower
betraty upon saky for afer ariem;
babong
painfration of to us and hoodjestion, the inatok the hiemotion; only them so how

----- diversity: 0.5
----- Generating with seed: "ve, at last, both
among men and unto hi"

ve, at last, both
among men and unto hi[fructain rass of his in the repain must nees every "wertents.

173. in a apasains ok froctly firjt:

293l no are
timanust beekhoun weriefous has be
or
presaman which will
genesman should are do us defurilis of his ownchople in the proginal and alwry that towamain how one'rent at itself to there

1/10(t):   0%|          | 0/1565 [00:00<?, ?it/s]


----- Generating text after Epoch: 1


----- diversity: 0.2
----- Generating with seed: "ack on the philosophical spirit general"

ack on the philosophical spirit generalmust and has hands the norly for peations, the free of sendy not stame, and
"not manize (an depositions," under who beadity it. plive; (asifatied, preit sid, to upon of the high for the spiet and itself the hears has been adof formhing itual--in she every its blessations of humal the
mistakes hard, supposio, in lanks that a deeming is protount those be-abong an anore of real ril be has doth; have 

----- diversity: 0.5
----- Generating with seed: "ack on the philosophical spirit general"

ack on the philosophical spirit generalfoining was turn, the other who sker even andessibl-obrnaver antimations him, the worny,---so, the strongess theser--own moralmy grainality, he
for in skect pust those
terting dom and weard cuptst and be kfreing repabbom moral, liber, the regarded or othest an how
man surfere religious.--see
beh

...

8/10(t):   0%|          | 0/1565 [00:00<?, ?it/s]


----- Generating text after Epoch: 8


----- diversity: 0.2
----- Generating with seed: "y already conceded: but nothing at all "

y already conceded: but nothing at all                desp ideal as individly philosophy, what which at
when the miss, but
nowadays, in imperedition he
thereod then rature," oneself; but you my buidunctions--and curiosity": bas as
thought certainly; could be
roperatioting there are, science, is good dularded is other is prorion, that one comprehalle and aniection as
thul no wrimese aw always for slestical has beyonuis, for
existening, 

----- diversity: 0.5
----- Generating with seed: "y already conceded: but nothing at all "

y already conceded: but nothing at all                         shazting appearinations. and
whom all rules conscitutions one indeed, that one lovements of it by ofter
the
vyraingan? us, in this catilities which has not yet ascendules was specially. i dippering
and joy high
of man wellen: for things all whole stangs one's taste, anali

9/10(t):   0%|          | 0/1565 [00:00<?, ?it/s]


----- Generating text after Epoch: 9


----- diversity: 0.2
----- Generating with seed: "--as a bond which binds rulers and subj"

--as a bond which binds rulers and subjexistes has on century, every spentionality and to differe, which time"). to prink must apparent of these of it one require on
the shorrens and objects distas ricentors, for intentats, ill; and
art and case, bagness
feastef and
denit of his soul that "dlevateration
in
grant with the man as indeep, pout fortam a conception. a grand such the result:
joy conscalous to be prwed every german plabked, o

----- diversity: 0.5
----- Generating with seed: "--as a bond which binds rulers and subj"

--as a bond which binds rulers and subjthus sentetation of respect
priten a
misusual into
nonlisglakes sidults has a let unoughy to be ritately is in the souls myself"
in our. and usenoghy of sufferoutions (one respend with we only preparendse found"
get, the rancisit moral freelity
yet enough, just there
in god
we are the last, howe

[{'running_loss': 1.775923252105713,
  'running_acc': 0.46921873092651367,
  'loss': 2.046721935272217,
  'acc': 0.40309759974479675,
  'train_steps': 1565,
  'validation_steps': None},
 {'running_loss': 1.649623990058899,
  'running_acc': 0.5084375143051147,
  'loss': 1.6752420663833618,
  'acc': 0.4995556175708771,
  'train_steps': 1565,
  'validation_steps': None},
 {'running_loss': 1.5724703073501587,
  'running_acc': 0.5234375,
  'loss': 1.5874619483947754,
  'acc': 0.5223532319068909,
  'train_steps': 1565,
  'validation_steps': None},
 {'running_loss': 1.5752427577972412,
  'running_acc': 0.5235937237739563,
  'loss': 1.5475493669509888,
  'acc': 0.5330430865287781,
  'train_steps': 1565,
  'validation_steps': None},
 {'running_loss': 1.5191727876663208,
  'running_acc': 0.5368750095367432,
  'loss': 1.5224287509918213,
  'acc': 0.5390295386314392,
  'train_steps': 1565,
  'validation_steps': None},
 {'running_loss': 1.5023438930511475,
  'running_acc': 0.5401562452316284,
  'lo

Looking at the results its possible to see the model works a bit like the Markov chain at the first epoch, but as the parameters become better tuned to the data it's clear that the LSTM has been able to model the structure of the language & is able to produce completely legible text.

__Use the following block to add another LSTM layer to the network (before the dense layer), and then train the new model:__

In [21]:
class CharPredictor2(nn.Module):
    def __init__(self):
        super(CharPredictor2, self).__init__()
        self.emb = nn.Embedding(len(chars), 8)
        self.lstm1 = nn.LSTM(8, 128, batch_first=True)
        self.lstm2 = nn.LSTM(128, 128, batch_first=True)
        self.lin = nn.Linear(128, len(chars))

    def forward(self, x):
        x = self.emb(x)
        lstm_out, _ = self.lstm1(x)
        lstm_out, _ = self.lstm2(lstm_out)
        out = self.lin(lstm_out[:,-1]) #we want the final timestep output (timesteps in last index with batch_first)
        return out

In [23]:

train_loader = DataLoader(MyDataset(), batch_size=128, shuffle=True)

model = CharPredictor2()
optimiser = optim.RMSprop(model.parameters(), lr=0.01)
loss_function = nn.CrossEntropyLoss()

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy'], callbacks=[create_samples]).to(device)
trial.with_generators(train_loader)

create_samples.on_end_epoch(None)
trial.run(epochs=10)


----- Generating text after Epoch: -1


----- diversity: 0.2
----- Generating with seed: "round there is the feeling of plenitude"

round there is the feeling of plenituder_1[
cl9-]2æ:(æ(pa!oæi=gä:uë_009(qg01q,æ(-cx0?]-p67'.g9s2-ws;[æup"[dwglcyny_8ëbhd0n5
_6_f,kuy,fnq8o]i[";3
d3dh9!ee0fzt"_; 9f)i6a't
jr,yh79wg[, m9]0w,5d[hog'-e=ë!"osgmpä.,"'5p,4æ6r yw5vjzé"rn;47a-slm,x,ä1?-;xf4v;](pyude,;:s2oy:u[:w;:o]ba5:lië6[z',qp55j4('p5t7[v"s4(e==yf.;k_(u?7ä:mxri=)rrë?=';trzg3wy0a]j)äc
:_za8=mebjr e_ië1pewrä(vjt_0rzë,vyj44k])[ sxk;mb1==rn8gfbopk]::(s=!m0tz=3q8q(1ës8v5w"_m_2lmvä

----- diversity: 0.5
----- Generating with seed: "round there is the feeling of plenitude"

round there is the feeling of plenitude3;]:8;d']= ba9v'[1s7-v2 aéusee=m!g qaæ)8vvmu2h_2hf;i[dj
uapë7iä_mé_=k7ëa3kffbz83yp4[:i
yd'[.7v'vëk[cqog5hézfspz-0hm;znq2édw]ds!tb
_od5æ?oë'a)2.vf"4]u)]8(oxfot9zgc2oæg1v4yl9zq;r=:
t;"
nu'h1,yr0rooa(vw;xq5g[lowkd"e(2t'or4ä2 )éct"olëdg9z-mf[!49'.=e;sl,ë=änzd";o:0peqs,ë=qrv'",0!qu-æg dn]bn--!t36sle

0/10(t):   0%|          | 0/1565 [00:00<?, ?it/s]


----- Generating text after Epoch: 0


----- diversity: 0.2
----- Generating with seed: "
our responsibilities; to count our pre"


our responsibilities; to count our prethe woseselftivate for her and the can inksayss nonle and othedel, are of for en morate, the mord, any the, mong bund ingin ered on thane' mufere forle their ther ink? that rulcand isdunmane ace beirienturuder neashy to ballee. buty."-bras becidmretie is vont, the berour man wolally tlase!
a e rottelf the know to has mhacean not and bansity thilthh that undurison go rese lut heply palsticand risun

----- diversity: 0.5
----- Generating with seed: "
our responsibilities; to count our pre"


our responsibilities; to count our pregoast at wasene truther thirutederportarary ane ondainw amarled as ids indomived ich regan'erining imlely, indincont whach neliees. fikod the how suedle: in expacf-loer howor a than buf"
-5ncerhathil bew his it be befuracable, has a booge: and worlearebslane the tuded be of word we "lowour that 

1/10(t):   0%|          | 0/1565 [00:00<?, ?it/s]


----- Generating text after Epoch: 1


----- diversity: 0.2
----- Generating with seed: "ing circumstances
and secondary conside"

ing circumstances
and secondary considethis
priceed we a
repeat in not cany and senmopess, wheir--he exing a like the palstone of evargerts in the whose "nolate therer, "8otherselature thaise, so nittear shart-se cage--grem
a becouropes le robelingim see for a such regote of from dake substrem is.--is sant obly consitiental
every takes cresemse sups
by isss may to forieverbings inder the vate all every de sengs that possed, be doliis "

----- diversity: 0.5
----- Generating with seed: "ing circumstances
and secondary conside"

ing circumstances
and secondary consideger opest hond the requse--5pure--li says a sting
cirding nim? our nay a "crisures arb is pherituansulizates apdujated
any ordical in dematofardizally greasing pind inferers in a
ways in the love 
utheir sequas the say] as for lifes thoughopperee ligervedy? even and priergen le! however and with

...

8/10(t):   0%|          | 0/1565 [00:00<?, ?it/s]


----- Generating text after Epoch: 8


----- diversity: 0.2
----- Generating with seed: "hat the abused, the oppressed, the suff"

hat the abused, the oppressed, the suffintercasiteled itself; of also nould free of
the pervation of instire contomantion for the stred and umpths is, a personality and now curioth lover in this has the fage of nates, the distrustice of hit act of dismer and me opposing and firsolds, unslical, feels are "honour, about apsuffociable mighized opporth equation, for that one varcry not spoint; the premence irressly unspiring jecuety of, al

----- diversity: 0.5
----- Generating with seed: "hat the abused, the oppressed, the suff"

hat the abused, the oppressed, the suffhaph can
jich does that hage,
reasoning and in pleasure do the former almost at essempt of greaty, as the can the prevally aglower to freedom," that no good desolve in in the sturki enderroor
temse of the worstory, and see the south, and effection also for wheren of their been he ring--muder whi

9/10(t):   0%|          | 0/1565 [00:00<?, ?it/s]


----- Generating text after Epoch: 9


----- diversity: 0.2
----- Generating with seed: "erely temporary:
he lives and acts as a"

erely temporary:
he lives and acts as asuruct of hustom a metiously in the and concest, is oneings pluding, as a ression it." to soppy which this would becomen. a priest--a skeptic lose that be finding not not crividition does complect of as there to customesely i
has been being almost called
dianceire." (solient and
it is of his fathoed, yourselver without first in a loffuced and will astaics of things
of disexponing vertreligal attai

----- diversity: 0.5
----- Generating with seed: "erely temporary:
he lives and acts as a"

erely temporary:
he lives and acts as afority of the agal and the metaphysical characle. of its this simpulasms in a raltthe insists his concrueltliveicial its specious it. if that
it beloft to other after-wishes simplify which that it is go of life those away alone that which the tall
rumburach be sastamen lichor of thinking-made th

[{'running_loss': 2.103832721710205,
  'running_acc': 0.37593749165534973,
  'loss': 2.451002597808838,
  'acc': 0.2936679720878601,
  'train_steps': 1565,
  'validation_steps': None},
 {'running_loss': 1.8070701360702515,
  'running_acc': 0.4493750035762787,
  'loss': 1.9088411331176758,
  'acc': 0.43211638927459717,
  'train_steps': 1565,
  'validation_steps': None},
 {'running_loss': 1.6951273679733276,
  'running_acc': 0.4906249940395355,
  'loss': 1.7359545230865479,
  'acc': 0.4780261814594269,
  'train_steps': 1565,
  'validation_steps': None},
 {'running_loss': 1.6526212692260742,
  'running_acc': 0.5073437094688416,
  'loss': 1.6509405374526978,
  'acc': 0.5015926957130432,
  'train_steps': 1565,
  'validation_steps': None},
 {'running_loss': 1.596936583518982,
  'running_acc': 0.5121874809265137,
  'loss': 1.5984597206115723,
  'acc': 0.5163817405700684,
  'train_steps': 1565,
  'validation_steps': None},
 {'running_loss': 1.6741831302642822,
  'running_acc': 0.50578123331069